In [1]:
import os
import cv2
from PIL import Image
import shutil

In [9]:
os.chdir('./darknet/data')
os.getcwd()

'C:\\Users\\RW\\darknet\\data'

In [34]:
def convert_box(classes, path, file_name):
    f_txt = open(path+'Label/'+file_name+'.txt')
    f_txt_new = open(path+'Label_New/'+file_name+'.txt', 'w')
    l = {}
    line = f_txt.readline()
    count = 0
    
    
    img = Image.open(path+'Images/'+file_name+'.jpg')
    width = img.size[0]
    height = img.size[1]
    
    while line:  
        line = line.strip('\n')
        line = line.split(' ')
        l[count] = line
        (x1, y1, x2, y2) = map(float, l[count][1:5])
        box_ratio = [(x2+x1)/(2*width), (y2+y1)/(2*height), (x2-x1)/width, (y2-y1)/height]
        box_ratio = ['{:.4f}'.format(i) for i in box_ratio] 
        l[count][1:5] = box_ratio
        l[count][0] = str(classes.index(l[count][0]))
        f_txt_new.write(" ".join(l[count]))
        f_txt_new.write('\n')
        line = f_txt.readline()
        count+=1
    f_txt.close()    
    f_txt_new.close()

In [32]:
def YOLO_format_image_parameter_list(aims, classes, ori_path):
    for aim in aims:
        for cls in classes:
            path = ori_path + aim + '/' + cls + '/'
            file_list = os.listdir(path+'Images/')
            for file_obj in file_list:
                print(path)
                file_name,file_extend = os.path.splitext(file_obj)
                convert_box(classes, path, file_name)

In [38]:
def file_list(aims, classes, ori_path):
    for aim in aims: 
        f = open(ori_path+'Image_Sets/'+aim+'.txt', 'w')
        for cls in classes:
            path = ori_path+aim+'/'+cls+'/'
            file_list = os.listdir(path+'Images/')
            for file_obj in file_list: 
                file_name,file_extend = os.path.splitext(file_obj)
                f.write(file_name+'\n')
        f.close()

In [46]:
def merge_documents(aims, classes, ori_path):
    for aim in aims:
        for cls in classes:
            path_ori = ori_path+aim+'/'+cls+'/Label_New/'
            path_dst = ori_path+aim+'/All_Labels/'
            file_list = os.listdir(path_ori)
            for file_obj in file_list:
                file_list_dst = os.listdir(path_dst)
                if file_obj in file_list_dst:
                    shutil.copyfileobj(open(path_ori+file_obj,'r'), open(path_dst+file_obj, 'a'))
                else:
                    shutil.copyfile(path_ori+file_obj, path_dst+file_obj)                 

In [65]:
def extract_redraw(classes, path, file_name):
    
    img = cv2.imread(path+'All_Images/'+file_name+'.jpg')
    height = img.shape[0]
    width = img.shape[1]
    
    f = open(path+'All_Labels/'+file_name+'.txt')
    line = f.readline()
    count = 0
    coordi = {}
    while line:
        line = line.strip('\n')
        line = line.split(' ')
        [a,b,c,d] = map(float, line[1:5])
        print((a,b,c,d))
        [x1, x2, y1, y2] = [int((a-c/2)*width), int((a+c/2)*width), int((b-d/2)*height), int((b+d/2)*height)]
        coordi[count] = [x1, x2, y1, y2]
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, classes[int(line[0])], (x1, y1), font, 2, (0,0,255), 5)
        line = f.readline()
        count += 1
    f.close()    
    
    cv2.imwrite(file_name+'_2.jpg', img)

In [7]:
aims = ['train', 'valid', 'test']
classes = ["Apple", "Orange"]
ori_path = 'Google Image/Dataset/'

In [ ]:
# Convert pixel-based object boundary coordinates to YOLO-recognizable format
YOLO_format_image_parameter_list(aims, classes, ori_path)
# Copy coordinate files from differnt classes (but for the same purpose, i.e. train/validation/test) into 
# the All_Labels folder.
# In rare cases the same image may exsit in two different classes. 
# In this case, the function will merge all boundary coordinates and object category into the same file.
merge_documents(aims, classes, ori_path)
# Double check if the multi-class coordinates are merged correctly.
extract_redraw(classes, 'Google Image/Dataset/test/', '410690d34ff17eb4')

In [70]:
# Revised voc_label.py
wd = os.getcwd().replace('\\', '/')

for aim in aims:

    image_ids = open('Google Image/Dataset/Image_Sets/%s.txt'%(aim)).read().strip().split()
    list_file = open('Google Image/absfilepath_%s.txt'%(aim), 'w')
    
    for image_id in image_ids:
        list_file.write('%s/Google Image/Dataset/%s/All_Images/%s.jpg\n'%(wd, aim, image_id))
    
    list_file.close()